In [1]:
%matplotlib inline 

import requests 
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations
#from __future__ import division
from math import log10,exp
import seaborn as sns
import scipy as sp
import sklearn
import sklearn.model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import csv

pd.options.display.mpl_style = 'default'

C:\Users\User\AppData\Local\conda\conda\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py:3267: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import warnings
import itertools
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import matplotlib

In [3]:
# load the dataset

dataload_num = 1
if dataload_num == 1:
    data = pd.read_csv('Solar/Home-electricity-data-v3.csv')
#end

In [ ]:
# Choose the data from household with certain post codes, i.e., 2259, 2261, ......
data_Postcode2259=data[data.Postcode == 2259]
data_Postcode2261=data[data.Postcode == 2261]
data_Postcode2262=data[data.Postcode == 2262]
data_Postcode2263=data[data.Postcode == 2263]
data_All4Postcodes=pd.concat([data_Postcode2259, data_Postcode2261,data_Postcode2262,data_Postcode2263], ignore_index=True)
#df.groupby(['Postcode', 'Customer'])
print(data_All4Postcodes.groupby(['Postcode', 'Customer']).count())

In [ ]:
# Adding day of week to separate weekday and weekend
import datetime as dt

data_All4Postcodes.date = data_All4Postcodes.date.apply(lambda x: dt.datetime.strptime(x, '%d/%m/%Y'))
data_All4Postcodes['day_of_week'] = data_All4Postcodes['date'].dt.dayofweek
data_All4Postcodes

In [ ]:
data_All4Postcodes_Consumption_GC=data_All4Postcodes[data_All4Postcodes.Consumption_Category == 'GC']
data_All4Postcodes_Consumption_GG=data_All4Postcodes[data_All4Postcodes.Consumption_Category == 'GG']
data_All4Postcodes_Consumption_GC

In [ ]:
HalfHourTimestamp = list(data_All4Postcodes_Consumption_GC.columns)[5:53]

Bin_Positions=[1,2864,5728,8592,11457,14321,17185,20050,22914]
Bin_Range_Values=[]
for x in range (48):
    data_All4Postcodes_Consumption_GC_temp=data_All4Postcodes_Consumption_GC.loc[:,HalfHourTimestamp[x]]
    Bin_Range_Values_tmp = [data_All4Postcodes_Consumption_GC_temp.nsmallest(Bin_Positions[y]).iloc[-1] for y in range (9)]
    Bin_Range_Values.append(Bin_Range_Values_tmp)
        #df11x2=df11.loc[:,'6:00']
        #df11x2r[x,y]=df11x2.nsmallest(pos[y]).iloc[-1]
Bin_Range_Values

In [ ]:
with open('Bin_Range_Values_halfhourlyv8.txt', 'w') as f:
    for item in Bin_Range_Values:
        f.write("%s\n" % item)

In [ ]:
DateTimestamp = pd.date_range(start = '2012-07-01', end = '2013-06-30', freq ="D")
dACG=data_All4Postcodes_Consumption_GC
dACG_Date=dACG[dACG.date == '2012-07-01']
dACG_Date_Time=dACG_Date.loc[:,'0:30']
dACG_Date_Time_Bins=dACG_Date_Time.groupby(pd.cut(dACG_Date_Time, Bin_Range_Values[0])).count().values.reshape(1, 8).T
dACG_Date_Time_Bins_temp=dACG_Date_Time_Bins
dACG_Date_Time_Bins_Noise= dACG_Date_Time_Bins + np.random.laplace(loc=0.0, scale=1.0, size=dACG_Date_Time_Bins.shape)
dACG_Date_Time_Bins_Noise=np.around((dACG_Date_Time_Bins_Noise/np.sum(dACG_Date_Time_Bins_Noise))*np.sum(dACG_Date_Time_Bins))
dACG_Date_Time_Bins_Noise_temp=dACG_Date_Time_Bins_Noise
dACG_Date_Time_Bins_Noise_temp
dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[0][1]+Bin_Range_Values[0][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[0][2]+Bin_Range_Values[0][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[0][3]+Bin_Range_Values[0][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[0][4]+Bin_Range_Values[0][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[0][5]+Bin_Range_Values[0][4])/2+dACG_Date_Time_Bins[5]*(Bin_Range_Values[0][6]+Bin_Range_Values[0][5])/2+dACG_Date_Time_Bins[6]*(Bin_Range_Values[0][7]+Bin_Range_Values[0][6])/2+dACG_Date_Time_Bins[7]*(Bin_Range_Values[0][8]+Bin_Range_Values[0][7])/4
dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[0][1]+Bin_Range_Values[0][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[0][2]+Bin_Range_Values[0][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[0][3]+Bin_Range_Values[0][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[0][4]+Bin_Range_Values[0][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[0][5]+Bin_Range_Values[0][4])/2+dACG_Date_Time_Bins_Noise[5]*(Bin_Range_Values[0][6]+Bin_Range_Values[0][5])/2+dACG_Date_Time_Bins_Noise[6]*(Bin_Range_Values[0][7]+Bin_Range_Values[0][6])/2+dACG_Date_Time_Bins_Noise[7]*(Bin_Range_Values[0][8]+Bin_Range_Values[0][7])/4
dACG_Date_Time_Bined_Accu_Elec_temp=dACG_Date_Time_Bined_Accu_Elec
dACG_Date_Time_Bined_Noise_Accu_Elec_temp=dACG_Date_Time_Bined_Noise_Accu_Elec
dACG_Date_Time_Bined_Noise_Accu_Elec_temp

In [ ]:

for x in DateTimestamp:
    dACG_Date=dACG[dACG.date == x]
    for y in range (48):
        #df11x2=df11.loc[:,HalfHourTimestamp1[x]]
        dACG_Date_Time=dACG_Date.loc[:,HalfHourTimestamp[y]]
        dACG_Date_Time_Bins=dACG_Date_Time.groupby(pd.cut(dACG_Date_Time, Bin_Range_Values[y])).count().values.reshape(1, 8).T

        dACG_Date_Time_Bins_Noise= dACG_Date_Time_Bins + np.random.laplace(loc=0.0, scale=1.0, size=dACG_Date_Time_Bins.shape)
        dACG_Date_Time_Bins_Noise=np.around((dACG_Date_Time_Bins_Noise/np.sum(dACG_Date_Time_Bins_Noise))*np.sum(dACG_Date_Time_Bins))

        dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2+dACG_Date_Time_Bins[5]*(Bin_Range_Values[y][6]+Bin_Range_Values[y][5])/2+dACG_Date_Time_Bins[6]*(Bin_Range_Values[y][7]+Bin_Range_Values[y][6])/2+dACG_Date_Time_Bins[7]*(Bin_Range_Values[y][8]+Bin_Range_Values[y][7])/4
        dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2+dACG_Date_Time_Bins_Noise[5]*(Bin_Range_Values[y][6]+Bin_Range_Values[y][5])/2+dACG_Date_Time_Bins_Noise[6]*(Bin_Range_Values[y][7]+Bin_Range_Values[y][6])/2+dACG_Date_Time_Bins_Noise[7]*(Bin_Range_Values[y][8]+Bin_Range_Values[y][7])/4

        #dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2
        #dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2

        #if y=='0:30':
        #    dfx4=dfx3
        #else:
        #dfx4=pd.concat([dfx4,dfx3], axis=1, ignore_index=True)
        dACG_Date_Time_Bins_temp=np.concatenate((dACG_Date_Time_Bins_temp,dACG_Date_Time_Bins),axis=1)
        dACG_Date_Time_Bins_Noise_temp=np.concatenate((dACG_Date_Time_Bins_Noise_temp,dACG_Date_Time_Bins_Noise),axis=1)
        dACG_Date_Time_Bined_Accu_Elec_temp=np.concatenate((dACG_Date_Time_Bined_Accu_Elec_temp,dACG_Date_Time_Bined_Accu_Elec),axis=0)
        dACG_Date_Time_Bined_Noise_Accu_Elec_temp=np.concatenate((dACG_Date_Time_Bined_Noise_Accu_Elec_temp,dACG_Date_Time_Bined_Noise_Accu_Elec),axis=0)
    
    #dfx5=pd.concat([dfx5,dfx4], axis=0, ignore_index=True)

In [ ]:
dACG_Date_Time_Bined_Accu_Elec_Final=dACG_Date_Time_Bined_Accu_Elec_temp[1:].reshape(17520, 1, order='C')
dACG_Date_Time_Bined_Noise_Accu_Elec_Final=dACG_Date_Time_Bined_Noise_Accu_Elec_temp[1:].reshape(17520, 1, order='C')

dACG_Date_Time_Bined_Accu_Elec_Final.shape

In [ ]:
with open('Bins_Infos.txt', 'w') as f:
    for item in dACG_Date_Time_Bins_temp:
        f.write("%s\n" % item)

In [ ]:
with open('dACG_Date_Time_Bined_Accu_Elec_Final.txt', 'w') as f:
    for item in dACG_Date_Time_Bined_Accu_Elec_Final:
        f.write("%s\n" % item)

In [ ]:
dACGG=data_All4Postcodes_Consumption_GG
dACG_groupby_Date_Sum=dACG.groupby(['date']).sum()
dACG_groupby_Date_Sum_temp=dACG_groupby_Date_Sum.loc[:,'0:30':'0:00']
new_rows = int(dACG_groupby_Date_Sum_temp.shape[1]) * dACG_groupby_Date_Sum_temp.shape[0]
new_cols = 1
dACG_Extact_Accu_Elec_Final=dACG_groupby_Date_Sum_temp.values.reshape(new_rows, new_cols, order='C')
dACG_Extact_Accu_Elec_Final
dACGG_groupby_Date_Sum=dACGG.groupby(['date']).sum()
dACGG_groupby_Date_Sum_temp=dACGG_groupby_Date_Sum.loc[:,'0:30':'0:00']
new_rows = int(dACGG_groupby_Date_Sum_temp.shape[1]) * dACGG_groupby_Date_Sum_temp.shape[0]
new_cols = 1
dACGG_Extact_Accu_Elec_Final=dACGG_groupby_Date_Sum_temp.values.reshape(new_rows, new_cols, order='C')
dACGG_Extact_Accu_Elec_Final

In [ ]:
dACGGMGC_Date_Time_Bined_Noise_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Date_Time_Bined_Noise_Accu_Elec_Final
dACGGMGC_Date_Time_Bined_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Date_Time_Bined_Accu_Elec_Final
dACGGMGC_Extact_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Extact_Accu_Elec_Final
dACGGMGC_Extact_Accu_Elec_Final

In [ ]:
dataset_Bined_Noise_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Date_Time_Bined_Noise_Accu_Elec_Final[:,0]})
dataset_Bined_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Date_Time_Bined_Accu_Elec_Final[:,0]})

dataset_Extact_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Extact_Accu_Elec_Final[:,0]})
dataset_Extact_Accu_Elec_Final2 = pd.DataFrame({'Column1':dACGG_Extact_Accu_Elec_Final[:,0]})

print(dataset_Extact_Accu_Elec_Final2)


In [ ]:
hourlyTimestamp = pd.date_range(start = '2012-07-01 00:30:00', end = '2013-07-01', freq ="30min")
hourlyTimestamp
dataset_Bined_Noise_Accu_Elec_Final.index=hourlyTimestamp
dataset_Bined_Accu_Elec_Final.index=hourlyTimestamp
dataset_Extact_Accu_Elec_Final.index=hourlyTimestamp
dataset_Extact_Accu_Elec_Final2.index=hourlyTimestamp
dataset_Bined_Noise_Accu_Elec_Final

In [ ]:
dataset_Bined_Noise_Accu_Elec_Final.to_csv('dataset_Bined_Noise_Accu_Elec_Finalv8.csv')
dataset_Bined_Accu_Elec_Final.to_csv('dataset_Bined_Accu_Elec_Finalv8.csv') 
dataset_Extact_Accu_Elec_Final.to_csv('dataset_Extact_Accu_Elec_Finalv8.csv') 

In [ ]:
#dataset1=dataset[['hour', 'weekday', 'day', 'week', 'Column1']]
dataset_Bined_Noise_Accu_Elec_Final_train = pd.DataFrame(data=dataset_Bined_Noise_Accu_Elec_Final, index=pd.date_range(start = '2012-12-01 00:30:00', end = '2013-02-03 00:00:00', freq ="30min")).dropna()
dataset_Bined_Noise_Accu_Elec_Final_test = pd.DataFrame(data=dataset_Bined_Noise_Accu_Elec_Final, index=pd.date_range(start ='2013-02-03 00:30:00', end ='2013-02-10 00:30:00',  freq ="30min")).dropna()
dataset_Bined_Accu_Elec_Final_test = pd.DataFrame(data=dataset_Bined_Accu_Elec_Final, index=pd.date_range(start ='2013-02-03 00:30:00', end ='2013-02-10 00:30:00',  freq ="30min")).dropna()
dataset_Extact_Accu_Elec_Final_test = pd.DataFrame(data=dataset_Extact_Accu_Elec_Final, index=pd.date_range(start ='2013-02-03 00:30:00', end ='2013-02-10 00:30:00',  freq ="30min")).dropna()
dataset_Extact_Accu_Elec_Final_test2 = pd.DataFrame(data=dataset_Extact_Accu_Elec_Final2, index=pd.date_range(start ='2013-02-03 00:30:00', end ='2013-02-10 00:30:00',  freq ="30min")).dropna()

dataset_Extact_Accu_Elec_Final_test2

In [8]:
dataset_Bined_Noise_Accu_Elec_Final_train = pd.DataFrame(data=dataset_All4Postcodes_SelectDate_Consumption_Net_Customer2_Final.Column1, index=pd.date_range(start = '2013-01-27 00:30:00', end = '2013-02-03 01:00:00', freq ="30min")).dropna()
dataset_Bined_Noise_Accu_Elec_Final_test = pd.DataFrame(data=dataset_All4Postcodes_SelectDate_Consumption_Net_Customer2_Final.Column1, index=pd.date_range(start ='2013-02-03 01:30:00', end ='2013-02-10 00:30:00',  freq ="30min")).dropna()


In [ ]:
#Plot observed and Predicted electricity value 

fig = plt.figure(figsize=(20,10))
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test.index, dataset_Bined_Noise_Accu_Elec_Final_test.Column1, label='Privatized quantized energy consumption with bins', color='b')
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test.index, dataset_Bined_Accu_Elec_Final_test.Column1, label='Quantized energy consumption with bins', color='k')
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test.index, dataset_Extact_Accu_Elec_Final_test.Column1, label='Actually energy consumption', color='r')
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test.index, dataset_Extact_Accu_Elec_Final_test2.Column1, label='Actually energy generation', color='g')

plt.legend(loc='upper right')

In [9]:
import warnings
import itertools
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import matplotlib

In [ ]:
p = d = q = range(0, 10)
pdq = list(itertools.product(p, d, q))

print('Examples of parameter combinations for ARIMA...')
print('SARIMAX: {}'.format(pdq[1]))
print('SARIMAX: {}'.format(pdq[2]))
print('SARIMAX: {}'.format(pdq[3]))
print('SARIMAX: {}'.format(pdq[4]))

In [ ]:
y = dataset_Bined_Noise_Accu_Elec_Final_train.values
y

In [ ]:
for param in pdq:
    #for param_seasonal in seasonal_pdq:
    try:
        mod = ARIMA(y,order=param)
        results = mod.fit()
        print('ARIMA{} - AIC:{}'.format(param, results.aic))
    except:
        continue

In [ ]:
pdq

In [ ]:
y = dataset_Bined_Noise_Accu_Elec_Final_train.values
predictions = list()
test=dataset_Bined_Noise_Accu_Elec_Final_test.values
for t in range(len(test)):
    try:
        model = ARIMA(y, order=(7, 0, 5))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        y=np.append(y[1:],obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    except:
        y_temp=y+np.random.laplace(loc=0.0, scale=1.0, size=y.shape)
        model = ARIMA(y_temp, order=(7, 0, 5))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        y=np.append(y[1:],obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [10]:
pdq_Selected=[(7, 0, 5),(4,0,4),(9, 0, 5),(8, 0, 4),(8, 0, 6),(6, 0, 5),(9, 0, 4)]

In [11]:
y = dataset_Bined_Noise_Accu_Elec_Final_train.values
test=dataset_Bined_Noise_Accu_Elec_Final_test.values
predictions = [None] * len(test)
for t in range(len(test)):
    for param in pdq_Selected:
        try:
            for p in range(15):
                try:
                    y_temp=y+np.random.laplace(loc=0.0, scale=(0.0+1*p), size=y.shape)
                    model = ARIMA(y_temp, order=param)
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    yhat = output[0]
                    predictions[t]=yhat
                    obs = test[t]
                    y=np.append(y[1:],obs)
                    print('predicted=%f, expected=%f' % (yhat, obs))
                except:             
                    continue
                else:
                    break
        except:             
            continue
        else:
            break

predicted=-0.180396, expected=-0.166000
predicted=-0.143321, expected=-0.167000
predicted=-0.223639, expected=-0.176000
predicted=-0.400347, expected=-0.183000
predicted=-0.173818, expected=-0.167000
predicted=-0.183206, expected=-0.168000
predicted=-0.119610, expected=-0.170000
predicted=-0.101030, expected=-0.184000
predicted=-0.168974, expected=-0.167000
predicted=-0.181471, expected=-0.164000
predicted=-0.146053, expected=-0.114000
predicted=-0.006570, expected=-0.158000
predicted=-0.182330, expected=-0.221000
predicted=-0.167242, expected=-0.103000
predicted=-0.101004, expected=-0.100000
predicted=-0.183773, expected=-0.010000
predicted=-0.041254, expected=0.318000
predicted=0.213681, expected=0.341000
predicted=0.177001, expected=0.251000
predicted=0.179627, expected=0.133000
predicted=0.034415, expected=0.141000
predicted=0.055997, expected=0.166000
predicted=-0.792877, expected=0.098000
predicted=0.027401, expected=0.005000
predicted=nan, expected=0.147000
predicted=0.116041, e

predicted=0.218318, expected=0.292000
predicted=0.279914, expected=0.278000
predicted=0.209445, expected=0.366000
predicted=0.327593, expected=0.270000
predicted=0.184294, expected=0.276000
predicted=0.248783, expected=0.218000
predicted=0.161975, expected=0.273000
predicted=0.227402, expected=0.303000
predicted=0.219336, expected=0.276000
predicted=0.167380, expected=0.249000
predicted=0.205784, expected=0.146000
predicted=0.029488, expected=0.131000
predicted=0.086119, expected=0.064000
predicted=-0.007584, expected=-0.007000
predicted=-0.054494, expected=-0.061000
predicted=-0.104232, expected=-0.196000
predicted=-0.227692, expected=-0.262000
predicted=-0.279975, expected=-0.302000
predicted=-0.305674, expected=-0.293000
predicted=-0.311400, expected=-0.309000
predicted=-0.345780, expected=-0.369000
predicted=-0.384814, expected=-0.657000
predicted=-0.621115, expected=-0.482000
predicted=0.000620, expected=-0.331000
predicted=-0.359136, expected=-0.313000
predicted=-0.352193, expect

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions)-1].index, dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions)-1].Column1, label='Privatized quantized energy consumption with bins', color='b')
#plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions1)-1].index, dataset_Bined_Accu_Elec_Final_test[:len(predictions1)-1].Column1, label='Quantized energy consumption with bins', color='k')
#plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions1)-1].index, dataset_Extact_Accu_Elec_Final_test[:len(predictions1)-1].Column1, label='Actually energy consumption', color='r')
#plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions)-1].index, predictions[1:len(predictions)], label='Prediction of privatized quantized energy consumption with bins', color='g')
plt.plot(dataset_Bined_Noise_Accu_Elec_Final_test[:len(predictions)-1].index, predictions[1:len(predictions)], label='Prediction of privatized quantized energy consumption with bins with filled data', color='y')

plt.legend(loc='upper right')

In [12]:
predictions1=np.concatenate(predictions, axis=0 )
predictions2=predictions1.reshape(-1,1)
predictions1

array([-1.80396000e-01, -1.43321023e-01, -2.23639199e-01, -4.00347455e-01,
       -1.73817696e-01, -1.83205631e-01, -1.19610064e-01, -1.01030161e-01,
       -1.68974087e-01, -1.81470962e-01, -1.46053206e-01, -6.57004689e-03,
       -1.82329841e-01, -1.67241777e-01, -1.01003949e-01, -1.83772592e-01,
       -4.12539096e-02,  2.13680676e-01,  1.77000514e-01,  1.79626653e-01,
        3.44147975e-02,  5.59966030e-02, -7.92876662e-01,  2.74012359e-02,
                   nan,  1.16041079e-01, -1.29554929e-01,  7.37515808e-02,
       -4.16633937e-02, -1.50260099e-01, -1.40254120e-02, -1.33319124e-01,
       -1.52839498e-01, -2.69664694e-01, -1.55250844e-01, -2.51465105e-01,
       -2.79717007e-01, -3.21395507e-01, -5.58309048e-01, -7.29084847e-01,
       -2.85873857e-01,             nan, -3.03269246e-01,             nan,
       -3.50967363e-01, -2.62774214e-01, -2.89459651e-01, -2.51742368e-01,
       -2.03888815e-01, -2.49622553e-01, -9.84869593e-02, -2.63987295e-01,
       -8.34019066e-02, -

In [24]:
MSE=np.sum(np.power((test - predictions3),2))/len(test)
MSE

0.0837726541211352

In [14]:
test - predictions2

array([[ 1.43960004e-02],
       [-2.36789770e-02],
       [ 4.76391994e-02],
       [ 2.17347455e-01],
       [ 6.81769645e-03],
       [ 1.52056307e-02],
       [-5.03899364e-02],
       [-8.29698393e-02],
       [ 1.97408723e-03],
       [ 1.74709621e-02],
       [ 3.20532057e-02],
       [-1.51429953e-01],
       [-3.86701586e-02],
       [ 6.42417770e-02],
       [ 1.00394886e-03],
       [ 1.73772592e-01],
       [ 3.59253910e-01],
       [ 1.27319324e-01],
       [ 7.39994856e-02],
       [-4.66266527e-02],
       [ 1.06585203e-01],
       [ 1.10003397e-01],
       [ 8.90876662e-01],
       [-2.24012359e-02],
       [            nan],
       [-1.45041079e-01],
       [ 1.96554929e-01],
       [-2.27515808e-02],
       [-1.01336606e-01],
       [ 2.06260099e-01],
       [-2.59745880e-02],
       [-4.26808763e-02],
       [-1.07160502e-01],
       [ 1.16664694e-01],
       [-4.67491564e-02],
       [-5.35348948e-02],
       [-8.82829934e-02],
       [-2.58604493e-01],
       [-3.1

In [16]:
predictions2[predictions2=='nan']=0

In [23]:
dataset_Prediction = pd.DataFrame({'Column1':predictions1})
dataset_Prediction.fillna(0, inplace=True)
predictions3=dataset_Prediction.values